#### Article outline

1. Demonstration of main Pandas methods
2. First attempt at predicting telecom churn
3. Reference

The main data structures in Pandas are implemented with `Series` and `DataFrame` classes. The former is a one-dimensional indexed array of some fixed data type. The latter is a two-dimensional data structure - a table - where each column contains data of the same type.

In [1]:
# load the required libraries
import numpy as np
import pandas as pd
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

We’ll demonstrate the main methods in action by analyzing a dataset on the churn rate of telecom operator clients. 
The data is sourced from `https://bigml.com/`. It can be downloaded from 

Let’s read the data (using read_csv), and take a look at the first 5 lines using the head method: